In [1]:
import sqlalchemy
from os import environ

import numpy as np
import pandas as pd

# from jupyter_dash import JupyterDash
# from dash import html
# from dash.dependencies import Input, Output
# import plotly.express as px
# from dash import dcc

In [2]:
engine = sqlalchemy.create_engine("mariadb+mariadbconnector://"+environ.get("USER")+\
                                  ":"+environ.get("PSWD")+"@127.0.0.1:3306/nba")

In [3]:
A_cols = ['A.Name', 'Matchup', 'Game_day', 'Result','MINS', 'PTS',
 'FGM', 'FGA', 'FGP', 'PM3', 'PA3', 'P3P', 'FTM', 'FTA','FTP',
 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'PF','Game_ID']

A_cols = ", ".join(A_cols)

In [4]:
fields = "* "

inner_join =  "Box_scores INNER JOIN Players on Box_scores.Player_ID = Players.ID) A "

inner_select = "(SELECT * FROM " + inner_join 

outer_join = "INNER JOIN Teams on Teams.ID=A.Team_ID "

outer_select = "SELECT " + A_cols +  ", Teams.Name as Team FROM " + inner_select  + outer_join

# "ORDER BY Game_day desc"

In [5]:
box_scores = pd.read_sql(outer_select,engine,parse_dates=["Game_day"])

In [6]:
percentages = box_scores.groupby("Team").sum()
percentages['FGP'] = 100*percentages['FGM']/percentages['FGA']
percentages['FTP'] = 100*percentages['FTM']/percentages['FTA']
percentages['P3P'] = 100*percentages['PM3']/percentages['PA3']
percentages['eFG'] = 100*(percentages['FGM']+.5*percentages['PM3'])/percentages['FGA']

In [7]:
percentages['POSS_kept'] = 100*percentages['OREB']/((percentages['FGA']-percentages['FGM'])+\
                                                    (percentages['FTA']-percentages['FTM']))

In [8]:
stat = 'POSS_kept'
s = percentages.sort_values(stat,ascending=False)
# s['Ranking'] = list(range(len(percentages),0,-1))
s['Ranking'] = list(range(1,len(percentages)+1))

In [9]:
s[[stat,'Ranking']]

,POSS_kept,Ranking
Team,,
MEM,24.626279,1
TOR,23.514851,2
NOP,21.852595,3
MIN,21.044090,4
BOS,21.028173,5
NYK,20.874862,6
IND,20.839161,7
MIA,20.795931,8
UTA,20.689655,9


In [ ]:
det = box_scores.groupby(["Team","Name"]).sum()

In [17]:
det = det.loc['DET',:]

In [18]:
det['FGP'] = 100*det['FGM']/det['FGA']
det['FTP'] = 100*det['FTM']/det['FTA']
det['P3P'] = 100*det['PM3']/det['PA3']
det['eFG'] = 100*(det['FGM']+.5*det['PM3'])/det['FGA']

In [19]:
det = det.round(1)

In [20]:
det["%OREB"] = 100*det["OREB"]/det["OREB"].sum()
det["%DREB"] = 100*det["DREB"]/det["DREB"].sum()
det["%PTS"] = 100*det["PTS"]/det["PTS"].sum()
det["%AST"] = 100*det["AST"]/det["AST"].sum()
det["%3"] = 100*det["PM3"]/det["PM3"].sum()
det["%FG"] = 100*det["FGM"]/det["FGM"].sum()
det["%FT"] = 100*det["FTM"]/det["FTM"].sum()
det["%STL"] = 100*det["STL"]/det["STL"].sum()
det["%TOV"] = 100*det["TOV"]/det["TOV"].sum()
det["%BLK"] = 100*det["BLK"]/det["BLK"].sum()
det["FTR"] = 100*det["FTA"]/det["FGA"]
det = det.round(1)

In [21]:
det

,MINS,PTS,FGM,FGA,FGP,PM3,PA3,P3P,FTM,FTA,...,%DREB,%PTS,%AST,%3,%FG,%FT,%STL,%TOV,%BLK,FTR
Name,,,,,,,,,,,,,,,,,,,,,
Cade Cunningham,1669,865,330,817,40.4,101,311,32.5,104,123,...,11.8,12.6,17.9,13.8,13.2,9.3,12.4,20.2,10.5,15.1
Cassius Stanley,155,52,19,46,41.3,4,17,23.5,10,10,...,0.7,0.8,0.3,0.5,0.8,0.9,1.0,0.7,0.6,21.7
Cheick Diallo,31,11,3,8,37.5,0,0,NaN,5,6,...,0.4,0.2,0.0,0.0,0.1,0.4,0.2,0.1,0.0,75.0
Cory Joseph,1350,437,152,334,45.5,53,127,41.7,80,92,...,5.5,6.4,13.5,7.3,6.1,7.2,7.2,8.3,5.1,27.5
Deividas Sirvydis,27,3,1,10,10.0,1,7,14.3,0,0,...,0.3,0.0,0.1,0.1,0.0,0.0,0.6,0.0,0.3,0.0
Derrick Walton Jr.,108,19,6,26,23.1,3,13,23.1,4,4,...,0.4,0.3,1.4,0.4,0.2,0.4,1.4,1.2,1.3,15.4
Frank Jackson,982,471,166,406,40.9,77,241,32.0,62,78,...,2.6,6.9,2.8,10.5,6.7,5.6,4.6,3.9,2.9,19.2
Hamidou Diallo,1265,636,261,526,49.6,23,93,24.7,91,140,...,9.9,9.3,4.8,3.1,10.5,8.1,14.1,6.4,5.4,26.6
Isaiah Livers,80,24,9,25,36.0,6,18,33.3,0,0,...,0.1,0.4,0.4,0.8,0.4,0.0,0.8,0.3,0.3,0.0


In [173]:
b = box_scores[['Team','FGA','FGM','PM3','PA3','Game_day']].groupby(["Team","Game_day"],as_index=False).mean()

In [91]:
c = b[['Team','Game_day']].groupby('Team').count()

In [100]:
means = box_scores[['Team','FGA','FGM','PM3','PA3']].groupby("Team").sum()

In [101]:
for i in means.columns:
    means[i] = means[i]/c['Game_day']
    means[i] = means[i].round(1)

In [102]:
means

,FGA,FGM,PM3,PA3
Team,,,,
ATL,87.8,41.1,12.6,33.5
BKN,88.6,41.2,11.2,31.8
BOS,87.4,39.9,12.6,36.6
CHA,92.2,42.2,13.8,38.4
CHI,87.2,42.2,11.1,29.5
CLE,84.4,39.4,11.7,33.1
DAL,85.7,39.2,12.8,36.9
DEN,86.5,41.0,13.0,36.7
DET,88.9,37.7,11.1,34.4
